In [0]:
from pyspark.sql.types import *
import pyspark.sql.functions as F
from elasticsearch import Elasticsearch, helpers
from pyspark.sql.functions import col, split, randn
from pyspark.sql.types import ArrayType, IntegerType
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import BinaryClassificationMetrics, MulticlassMetrics
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np 
from pyspark.mllib.random import RandomRDDs
from pyspark.sql.window import Window

ES_HOST= '10.0.0.20' #'da2020w-0016.eastus.cloudapp.azure.com' # VM Server 
es = Elasticsearch([{'host': ES_HOST}], timeout=60000)

In [0]:
source_dest_df = spark.read.csv('source_dest_df_1.csv',header=True)
closest_stops_df = spark.read.csv('atrractions_with_busStops.csv',header=True)

In [0]:
schema_structfields = [
                      StructField("_id", MapType(StringType(), StringType(), True),True), 
                       StructField("actualDelay",LongType(),True),
                       StructField("angle",DoubleType(),True),
                       StructField("anomaly",BooleanType(),True),
                       StructField("areaId",LongType(),True),
                       StructField("areaId1",LongType(),True),
                       StructField("areaId2",LongType(),True),
                       StructField("areaId3",LongType(),True),
                       StructField("atStop",BooleanType(),True),
                       StructField("busStop",LongType(),True),
                       StructField("calendar",MapType(StringType(), StringType(),True),True),
                       StructField("congestion",BooleanType(),True),
                       StructField("currentHour",LongType(),True),
                       StructField("dateType",LongType(),True),
                       StructField("dateTypeEnum",StringType(),True),
                       StructField("delay",LongType(),True),
                       StructField("direction",LongType(),True),
                       StructField("distanceCovered",DoubleType(),True),
                       StructField("ellapsedTime",LongType(),True),
                       StructField("filteredActualDelay",LongType(),True),
                       StructField("gridID",StringType(),True),
                       StructField("journeyPatternId",StringType(),True), 
                       StructField("justLeftStop",BooleanType(),True),
                       StructField("justStopped",BooleanType(),True),
                       StructField("latitude",DoubleType(),True), 
                       StructField("lineId",StringType(),True), 
                       StructField('loc',StructType([StructField('coordinates',ArrayType(DoubleType(),True),True),StructField('type',StringType(),True)]),True),
                       StructField("longitude",DoubleType(),True),  
                       StructField("poiId",LongType(),True),
                       StructField("poiId2",LongType(),True),
                       StructField("probability",DoubleType(),True),
                       StructField("systemTimestamp",DoubleType(),True),
                       StructField("timestamp",MapType(StringType(),StringType(),True)), 
                       StructField("vehicleId",LongType(),True),
                       StructField("vehicleSpeed",LongType(),True)]

schema = StructType(schema_structfields)
big_df = spark.read.json('/mnt/dacoursedatabricksstg/dacoursedatabricksdata/busFile', schema=schema)

#display(es_big_df)

In [0]:
fixed_df = big_df.withColumn('_id', F.map_values(big_df._id)[0])\
                  .withColumn('calendar', F.map_values(big_df.calendar)[0])\
                  .withColumn('timestamp', F.map_values(big_df.timestamp)[0])\
                  .withColumn('loc', big_df['loc']['coordinates'])\
                  .withColumnRenamed('_id', 'id')
                     
# fix UTX to timestamp- relevant columns: timestamp, calender, systemTimestamp 
fixed_df = fixed_df.withColumn('timestamp', F.to_timestamp(F.from_unixtime(fixed_df.timestamp / (1000))))
fixed_df = fixed_df.withColumn('calendar', F.to_timestamp(F.from_unixtime(fixed_df.calendar / (1000000))))
fixed_df = fixed_df.withColumn('systemTimestamp', F.to_timestamp(F.from_unixtime(fixed_df.systemTimestamp)))
fixed_df = fixed_df.withColumn('mins_delay', fixed_df.delay / 60).withColumn('mins_actualDelay', fixed_df.actualDelay / 60) 
fixed_df=fixed_df.withColumn('realHour', F.hour(fixed_df.timestamp))
fixed_df=fixed_df.withColumn("isWeekend", F.dayofweek(fixed_df.timestamp).isin([1,7]).cast("int"))
fixed_df = fixed_df.drop('anomaly', 'probability', 'direction', 'poiId', 'poiId2', 'longitude', 'latitude', 'calendar', 'currentHour', 'dateType', 'dateTypeEnum','filteredActualDelay', 'actualDelay') 
fixed_df=fixed_df.withColumn('month', F.month(fixed_df.timestamp))
fixed_df=fixed_df.withColumn('year', F.year(fixed_df.timestamp))
fixed_df=fixed_df.withColumn('date', F.to_date(fixed_df.timestamp))
fixed_df=fixed_df.withColumn("season" ,F.expr(
    """IF (month IN (3,4,5) ,"Spring", IF (month IN (6,7,8), "Summer" ,IF (month IN (9,10,11) ,"Autumn","Winter")))"""
))
fixed_df=fixed_df.withColumn("timeInDay" ,F.expr(
    """IF (realHour IN (5,6,7,8,9,10,11) ,"Morning", IF (realHour IN (12, 13, 14, 15, 16, 17), "Afternoon" ,IF (realHour IN (18, 19, 20) ,"Evening","Night")))"""
))
fixed_df=fixed_df.withColumn("isInCenter" ,F.expr(
    """IF (busStop IN (278, 281, 4724, 274, 279, 4725, 4508, 272, 277, 270, 6059, 271, 7402, 1184, 4717, 288, 289, 7591, 299, 298, 297, 302, 301, 273, 315, 7622, 334, 335, 336, 340, 317, 319, 325, 7392, 328, 345, 346, 7588, 404, 405, 406, 320, 1359, 7582, 1358, 7581, 1279, 1278, 4522, 4521, 494, 495, 792, 793) ,true, false)"""
))

#Drop duplicates & Null rows
fixed_df = fixed_df.dropDuplicates()
es_big_df = fixed_df.na.drop(how="all")

In [0]:
es_big_df = es_big_df.withColumn('line_num', es_big_df['journeyPatternId'][0:4])
es_big_df = es_big_df.withColumn('line_num', F.regexp_replace('line_num', '^0+', '')) 
es_big_df = es_big_df.withColumn('direction', es_big_df['journeyPatternId'][5:5][0:1])

In [0]:
first_joined_df=es_big_df.join(source_dest_df, on=["line_num", 'direction'],how="left")
display(first_joined_df) 

In [0]:
closest_stops_df=closest_stops_df["Name","AddressLocality","distance","rank","busStop"]

In [0]:
joined_df=first_joined_df.join(closest_stops_df,on=["busStop"],how="left")

In [0]:
joined_df = joined_df.withColumn("attraction_catagory",F.when(F.col('atStop')==False,2).otherwise(F.when(F.col('Name').isNull(),0).otherwise(1)))
display(joined_df) 

In [0]:
task_2_df = joined_df[joined_df.atStop==True]

In [0]:
task_2_df=task_2_df["mins_delay","areaId1","congestion","loc","isInCenter","line_num","source","dest","attraction_catagory","year"]

In [0]:
# change congestion to binary 
task_2_df=task_2_df.withColumn('binary_congestion', F.expr("""IF (congestion=True, 1, 0)""")).withColumn('binary_isInCenter', F.expr("""IF (isInCenter=True, 1, 0)"""))
# split col to longitude and latitude 
task_2_df=task_2_df.withColumn('longitude', F.col("loc")[0]).withColumn('latitude', F.col("loc")[1]) ###todo- is correct way?
task_2_df=task_2_df.drop("loc","congestion","isInCenter")
# change null values to 'empty'
task_2_df = task_2_df.withColumn("source", F.when(F.col('source').isNull() ,"empty").otherwise(F.col('source'))).withColumn("dest", F.when(F.col('dest').isNull() ,"empty").otherwise(F.col('dest')))



In [0]:
def prepare_data_for_lr():
  stages = []
  
  categoricalColumns = ['binary_congestion', 'areaId1', 'line_num','source','dest','attraction_catagory']
  numericCols = ['mins_delay', 'longitude', 'latitude']
  
  for categoricalCol in categoricalColumns:
      stringIndexer = StringIndexer(inputCol = categoricalCol, outputCol = categoricalCol + 'Index').setHandleInvalid("keep")
      stages += [stringIndexer]


  label_stringIdx = StringIndexer(inputCol = 'binary_isInCenter', outputCol = 'label').setHandleInvalid("keep")
  stages += [label_stringIdx]


  assemblerInputs = [c + "Index" for c in categoricalColumns] + numericCols
  assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
  stages += [assembler]
  
  return stages 

In [0]:
def pipeline_prepare_data(stages, input_df, is_delay=False):
  pipeline = Pipeline(stages = stages)
  pipelineModel = pipeline.fit(input_df)
  df = pipelineModel.transform(input_df)
  selectedCols = ['label', 'features'] + input_df.columns
  df = df.select(selectedCols)
  return df 

In [0]:
def create_lr_model(train):
  lr = LogisticRegression(featuresCol = 'features', labelCol = 'label', maxIter=10)
  lrModel = lr.fit(train)
  return lrModel

In [0]:
def predict(lrModel, test):
  predictions = lrModel.transform(test)
  predictions.groupBy('label', 'prediction').count().show()
  return predictions 

In [0]:
stages = prepare_data_for_lr()
#split to train 
train_df=task_2_df[task_2_df["year"]== 2017]
df_after_pipeline = pipeline_prepare_data(stages, tain_df)
#Logistic Regression Model
lrModel = create_lr_model(df_after_pipeline)

In [0]:
lrModel.save("lrm_model_task3.model")

In [0]:
from pyspark.sql.functions import row_number, monotonically_increasing_id
from pyspark.sql import Window


def from_catCol_to_index(train_df,col_name):
  counts = train_df.select(col_name).groupBy(col_name).count().sort(col("count").desc())
  counts =counts.withColumn(col_name+"_index",row_number().over(Window.orderBy(monotonically_increasing_id()))-1)
  counts =counts.withColumn(col_name+"_index",F.col(col_name+"_index").cast(IntegerType()))

  counts=counts.select([col_name+"_index",col_name])
  counts.write.csv(col_name+'_index.csv',header=True)
  return counts


col_names=["line_num",'source','dest']
for col_name in col_names:
  counts=from_catCol_to_index(train_df,col_name)